In [8]:
%matplotlib inline
from fenparsev4 import *
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.tools.shortcuts import buildNetwork
from pybrain.structure.modules import TanhLayer
import numpy as np
import random
from sklearn.metrics import confusion_matrix
import pickle

Position(board='         \n         \n rnbqkbnr\n pppp.ppp\n ........\n ....p...\n ....P...\n .....N..\n PPPP.PPP\n RNBQKB.R\n         \n         ', score=0, wc=(True, True), bc=(True, True), ep=0, kp=0)
(81, 44)
(32, 33)
(76, 55), N->p


In [9]:
with open("../data/2015-12-01-fen_sample") as f:
    content = [" ".join(line.split(" ")[:2]) for line in f.readlines()]

In [10]:
df = fen2mlinput(content[:1000])

In [11]:
length = df.shape[0]
df = df.set_index([range(0,length)])
#shuffle
df = df.reindex(np.random.permutation(df.index))

In [12]:
a = [1,2,3,4]
a[-1:]

[4]

In [13]:
all_train = df.iloc[:9000, :]
# all_train = all_train[[s.piece_moved.islower() for s in all_train['move_info']]]
# all_train = all_train[[s.piece_moved.isupper() for s in all_train['move_info']]]
all_test = df.iloc[-1000:, :]
# all_test = all_test[[s.piece_moved.islower() for s in all_test['move_info']]]
#all_test = all_test[[s.piece_moved.isupper() for s in all_test['move_info']]]
X_train = list(all_train['x'])
X_test = list(all_test['x'])
y_train = [[1] if y == 1 else [0] for y in all_train['y']]
y_test = [1 if y == 1 else 0 for y in all_test['y']]

In [ ]:
y_train

In [14]:
class Experiment:
    def __init__(self, net, train_df, test_df, num_epochs, verbose=True):
        self.net = net
        self.train_df = train_df
        self.test_df = test_df
        self.train_ds = self.build_dataset(train_df)
        self.test_ds = self.build_dataset(test_df)
        self.num_epochs = num_epochs
        self.verbose = verbose
        
    #takes in full dataframe and converts to pybrain dataset
    def build_dataset(self, df):
        ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
        for i in df.index:
            ds.addSample(df.loc[i,'x'], df.loc[i,'y'])
        ds._convertToOneOfMany(bounds=[0, 1])
        return ds

    #adds specific piece confusion matrices to results dict d
    def add_piece_specifics(d):
        pass
        
    
    def run_experiment(self):
        if self.verbose:
            print "Training network"
        trainer = BackpropTrainer(self.net,  self.train_ds, self.verbose)
        print "Built network"
        y_pred = []

        trainer.trainEpochs(self.num_epochs)
        prob_out = net.activateOnDataset(self.test_ds)
        y_pred = prob_out.argmax(axis=1)  # the highest output activation gives the class
        #out = out.reshape(X_train.shape)
        #print(y_pred)
        #print(prob_out)
        cm = confusion_matrix(y_test, y_pred)
        print(cm)
    #    print(confusion_matrix(y_train, out))
        return pd.DataFrame({"train_size": self.train_df.shape[0], 
                            "train_white_count" : sum([1 if a.isupper() else 0 for a in self.train_df['piece_moved']]),
                            "confusion_matrix" : [cm],
                            "accuracy": [(cm[0][0] + cm[1][1]) * 1.0 / (sum([sum(c) for c in cm]))]})

In [17]:
from pybrain.structure import FeedForwardNetwork, RecurrentNetwork
from pybrain.structure import LinearLayer, SigmoidLayer
from pybrain.structure import FullConnection

def buildSimpleNet(hidden_layers, output_layers, bias, hidden_class):
    return buildNetwork(1536, hidden_layers, output_layers, bias, hiddenclass=hidden_class)

def buildFeedForward():
    n = FeedForwardNetwork()
    
    inLayer = LinearLayer(1536)
    hiddenLayer = SigmoidLayer(3)
    outLayer = LinearLayer(2)
    
    n.addInputModule(inLayer)
    n.addModule(hiddenLayer)
    n.addOutputModule(outLayer)
    
    in_to_hidden = FullConnection(inLayer, hiddenLayer)
    hidden_to_out = FullConnection(hiddenLayer, outLayer)
    
    n.addConnection(in_to_hidden)
    n.addConnection(hidden_to_out)
    
    n.sortModules()
    return n
def buildBigFeedForward():
    n = FeedForwardNetwork()
    
    inLayer = LinearLayer(1536)
    hiddenLayer = SigmoidLayer(100)
    hiddenLayer2 = LinearLayer(50)
    hiddenLayer3 = SigmoidLayer(10)
    outLayer = LinearLayer(2)
    
    n.addInputModule(inLayer)
    n.addModule(hiddenLayer)
    n.addModule(hiddenLayer2)
    n.addModule(hiddenLayer3)
    n.addOutputModule(outLayer)
    
    in_to_hidden = FullConnection(inLayer, hiddenLayer)
    hidden_to_hidden2 = FullConnection(hiddenLayer, hiddenLayer2)
    hidden2_to_hidden3 = FullConnection(hiddenLayer, hiddenLayer3)

    hidden_to_out = FullConnection(hiddenLayer3, outLayer)
    
    n.addConnection(in_to_hidden)
    n.addConnection(hidden_to_hidden2)
    n.addConnection(hidden2_to_hidden3)

    n.addConnection(hidden_to_out)
    
    n.sortModules()
    return n
def buildRNN():
    n = RecurrentNetwork()
    
    n.addInputModule(LinearLayer(1536, name='in'))
    n.addModule(SigmoidLayer(3, name='hidden'))
    n.addOutputModule(LinearLayer(2, name='out'))
    
    n.addConnection(FullConnection(n['in'], n['hidden'], name='c1'))
    n.addConnection(FullConnection(n['hidden'], n['out'], name='c2'))
    
    n.addRecurrentConnection(FullConnection(n['hidden'], n['hidden'], name='c3'))
   
    n.sortModules()
    
    return n

In [21]:
# results_df = pd.DataFrame()
# for train_instances in [10, 500, 1000, 2000, 3000, 4000, 5000]:
#     e = Experiment()
#     results_df = results_df.append(e.run_experiment(all_train[:train_instances], all_test, 1))
net = buildNetwork(1536, 50, 2, bias=True, hiddenclass=TanhLayer)
#net = buildSimpleNet(2, 2, True, TanhLayer)
#net = buildFeedForward()
#net = buildRNN()
#net = buildBigFeedForward()
e = Experiment(net, all_train, all_test, 1)
results_df = e.run_experiment()

Training network
Built network
[[481   0]
 [519   0]]


In [19]:
results_df

,accuracy,confusion_matrix,train_size,train_white_count
0,0.481,"[[481, 0], [519, 0]]",9000,4344


In [30]:
#[1 if a.isupper() else 0 for a in all_train['piece_moved']]
df.head()

,move,piece_moved,piece_captured,preboard,postboard,x,y
9690,"(88, 68)",p,.,"[ RNBKQBNR\n, PPP.PPPP\n, ........\n, ...P....","([ RNBKQBNR\n, PPP.PPPP\n, ........\n, ...P...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",0
402,"(71, 81)",R,.,"[ r.....k.\n, pprn.p.q\n, ..p.bBp.\n, .PPpP...","([ r.....k.\n, pp.n.prq\n, ..p.bBp.\n, .PPp...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
6565,"(86, 85)",q,.,"[ .K.R.BNR\n, PPP.P.PP\n, ........\n, ........","([ .K.R.BNR\n, PPP.P.PP\n, ........\n, .......","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
915,"(94, 95)",R,.,"[ .....rk.\n, .....pp.\n, ..p.q.np\n, .pQ.P...","([ .....rk.\n, .....pp.\n, ..p.q.np\n, .pQ....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
2743,"(75, 76)",R,.,"[ ..r...k.\n, .p.R..bp\n, r...p.p.\n, p...P...","([ ......k.\n, .p.R..bp\n, r...p.p.\n, p......","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [9]:
#train a simple neural network on 2000 instances
print "Starting"
# build dataset
train_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
for i in range(0,500):
    #j = random.randint(0,2000)
    train_ds.addSample(X_train[i], y_train[i])
    
test_ds = ClassificationDataSet(1536, nb_classes=2, class_labels=["Illegal", "Legal"])
for i in range(0,500):
    test_ds.addSample(X_test[i], y_test[i])
print "Building dataset"
#print(train_ds.getField('target'))
train_ds._convertToOneOfMany(bounds=[0, 1])
test_ds._convertToOneOfMany(bounds=[0, 1])
#print(train_ds.getField('target'))
print "Converted"
net     = buildNetwork(1536, 100, 2, bias=True, hiddenclass=TanhLayer)
trainer = BackpropTrainer(net,  train_ds, verbose=True)
print "Built network"
y_pred = []
for i in range(1):
    trainer.trainEpochs( 1 )
    prob_out = net.activateOnDataset(test_ds)
    y_pred = prob_out.argmax(axis=1)  # the highest output activation gives the class
    #out = out.reshape(X_train.shape)
    print(y_pred)
    #print(prob_out)
    print(confusion_matrix(y_test, y_pred))
#    print(confusion_matrix(y_train, out))


Starting
Building dataset
Converted
Built network
Total error: 8.2274069558
[1 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 0 1
 0 0 1 0 0 1 1 1 1 0 0 0 1 0 1 1 1 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 1 0 0 1
 0 0 1 1 1 1 0 1 0 0 0 0 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 0 1 0 1 0 0 1 1
 1 1 0 0 1 1 1 0 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 1 1 1 1 1 0 1 1 1 1 0 0 0 1
 0 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 1 0 1 1 0 0 0 1 1 0 0 1 0 1 1 0 1 1 0
 1 1 1 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 0 1 0 1 0
 1 0 1 1 0 0 1 1 0 0 1 0 0 1 0 1 1 0 0 0 1 0 0 0 1 0 1 0 1 1 0 0 0 1 1 0 0
 1 1 1 1 1 1 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 1 1 1 0 0 1 1 1 1 1 0 0 0
 1 0 1 1 0 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 0 1 0 0 0 0 1 1 0 1 1 1 0 1 1 1 1
 0 0 1 0 1 0 0 0 1 1 0 1 1 1 1 0 0 0 1 1 1 1 0 0 1 0 0 1 1 1 1 1 1 1 0 1 1
 0 0 0 1 0 1 1 1 1 0 1 0 1 1 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1 1 0 1 1 0 1
 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 1 0 0 1 1 1 0 0
 0 0 1 0 0 1 1 1 0 0 1 1

In [10]:
y_true = pd.Series(y_test)
y_pred = pd.Series(y_pred)
pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,125,119,244
1,119,137,256
All,244,256,500


In [11]:
y_pred = pd.Series(y_pred)
length = all_test.shape[0]
all_test = all_test.set_index([range(0,length)])
all_test.loc[:,'predicted'] = y_pred

In [12]:
y_pred.head()

0    1
1    1
2    0
3    0
4    1
dtype: int64

In [13]:
all_test = all_test.iloc[:500,:]

In [15]:
def collect_results(test_df, predicted_y)
    result_df = pd.DataFrame(columns=['training_instances', 'testingpct_white_moves',])
    
    #add predicted
    length = all_test.shape[0]
    test_df = all_test.set_index([range(0,length)])
    test_df.loc[:,'predicted'] = y_pred
    
    #calculate overall confusion matrix
    cm = confusion_matrx(test_df['y'], predicted_y)
    
    #calculate each piece confusion matrix
    for p in "pPrRnNbBqQkK":
        specific_piece = all_test[all_test['piece_moved'] == p]
#         print(p)
        cm = confusion_matrix(specific_piece['y'], specific_piece['predicted'])
        test_df.loc[:, p + '_perf'] = cm
#         print(cm)
#         print(1.0 * cm[0][0] / (sum([sum(a) for a in cm])))

p
[[36 20]
 [29 16]]
0.356435643564
P
[[13 23]
 [16 23]]
0.173333333333
r
[[ 5  7]
 [ 8 10]]
0.166666666667
R
[[14  6]
 [12 12]]
0.318181818182
n
[[17  4]
 [11  6]]
0.447368421053
N
[[ 4 11]
 [ 4 12]]
0.129032258065
b
[[6 3]
 [8 8]]
0.24
B
[[ 5 11]
 [ 2 16]]
0.147058823529
q
[[11 10]
 [ 6 11]]
0.289473684211
Q
[[ 9  8]
 [ 7 10]]
0.264705882353
k
[[2 9]
 [8 6]]
0.08
K
[[3 7]
 [8 7]]
0.12
